This notebook demonstrates running the post-process phase, in isolation.

It assumes the intermediate input has been saved already. See notes in "read input files" below.

This was done for performance testing: I wanted a way to run "just this part" without needing to redo the rather expensive prediction phase. (E.g. the GPUs aren't working on my intel mac laptop)

# Setup

In [1]:
import os
import timeit

In [2]:
# Parameters (overridden by notebook parameters)
raw_predictions_path = './saves/raw_segmentation_predictions.npz'
model_path = os.path.expanduser('~') + '/.keras/models/MultiplexSegmentation'
output_path = './saves'

### Load Mesmer model & create app

In [3]:
from imaging_helpers import make_app

app = make_app(model_path=model_path)

2023-10-24 15:08:09.673697: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Loaded model in 27.350531013013097 s


### Read input files

The `output_images` file is a bit unlike all other saved & loaded npz files in these notebooks. It's generated by inserting this code:

```
        import numpy as np
        self.logger.debug('Saving segmentation image')
        np.savez_compressed('raw_segmentation_predictions.npz', output_images=output_images)

        # Code right after this is:
        # Postprocess predictions to create label image
        # label_image = self._postprocess(output_images, **postprocess_kwargs)
```

into the [base application prediction method](https://github.com/vanvalenlab/deepcell-tf/blob/259f35f574290a728cae966cb6f812cdccc2ab38/deepcell/applications/application.py#L393), right before the postprocess call.

Then, in your `deepcell-imaging` checkout–where you're running this notebook in `jupyter-lab`–change your virtual environment to be a symlink to your `deepcell-tf` checkout. Something like this:

`$venv_path/lib/python3.10/site-packages/deepcell -> $deepcell_tf_checkout/deepcell/`

I think there are better ways to set up python environments with "editable" installs but ... another problem for another day.

In [4]:
import numpy as np

t = timeit.default_timer()

with np.load(raw_predictions_path, allow_pickle=True) as loader:
    output_images = loader['output_images']

# this is getting persisted weirdly maybe because it has a dict in it
# Doing this transform extracts the single element that was deserialized,
# which is a tuple containing (), and the actual images. So we select the
# second item of that tuple which at least is the output images we serialized.
output_images = list(np.ndenumerate(output_images))[0][1]

print('Loaded output_images in %s s' % (timeit.default_timer() - t))
print('output_images has dimensions: %s' % (output_images['whole-cell'][0].shape,))

Loaded output_images in 13.6647123080038 s
output_images has dimensions: (1, 9777, 14239, 1)


### Run post-process

In [5]:
from deepcell.applications.mesmer import build_postprocessing_kwargs

t = timeit.default_timer()

# build_postprocessing_kwargs depends on a PR I haven't submitted to vanvalenlab yet. :(
postprocess_kwargs = build_postprocessing_kwargs()

profiling = 'speedscope'
speedscope_output_path = 'speedscope.json'

match profiling:
  case 'snakeviz':
    import snakeviz
    %load_ext snakeviz
    %snakeviz -t label_image = app._postprocess(output_images, **postprocess_kwargs)
  case 'speedscope':
    import speedscope
    with speedscope.track(speedscope_output_path):
        label_image = app._postprocess(output_images, **postprocess_kwargs)
  case _:
    label_image = app._postprocess(output_images, **postprocess_kwargs)

print('Post-process finished in %s s' % (timeit.default_timer() - t))

# To open new tab on snakeviz profiling,
# $ snakeviz /var/folders/nz/qx4dxfys0_l2tn57sz_dyn440000gn/T/tmp645j9al3
# where you use the path from the log: Profile stats marshalled to file '...'


/Users/davidhaley/dev/deepcell-imaging/notebooks/venv/lib/python3.10/site-packages/deepcell_toolbox/deep_watershed.py:108: UserWarning: h_maxima peak finding algorithm was selected, but the provided image is larger than 5k x 5k pixels.This will lead to slow prediction performance.
  warnings.warn('h_maxima peak finding algorithm was selected, '


Post-process finished in 285.6167378560058 s
